# **Pytorch**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torchsummary import summary

class custom_pytorch(nn.Module):
    def __init__(self, num_classes):
        super(custom_pytorch, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 256 * 14 * 14)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
# Create an instance of the model
num_classes = 10
model = custom_pytorch(num_classes)
#model.load_state_dict(torch.load('/kaggle/working/custom.pth'))

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 51.8M


In [2]:
torch.save(model.state_dict(), 'custom.pth')

# **Tensorflow**

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

def custom_tensorflow(num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)  # Layer 1
    x = layers.BatchNormalization()(x)  # Layer 2
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 3
    
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)  # Layer 4
    x = layers.BatchNormalization()(x)  # Layer 5
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 6
    
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)  # Layer 7
    x = layers.BatchNormalization()(x)  # Layer 8
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 9
    
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)  # Layer 10
    x = layers.BatchNormalization()(x)  # Layer 11
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 12
    
    x = layers.Flatten()(x)  # Layer 13
    x = layers.Dropout(0.5)(x)  # Layer 14
    x = layers.Dense(1024, activation='relu')(x)  # Layer 15
    x = layers.Dropout(0.5)(x)  # Layer 16
    outputs = layers.Dense(num_classes, activation='softmax')(x)  # Layer 17
    
    model = models.Model(inputs, outputs)
    return model

# Create an instance of the model
num_classes = 10
tf_model = custom_tensorflow(num_classes)

n_parameters = tf_model.count_params()
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

2024-06-04 12:28:33.650886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 12:28:33.650987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 12:28:33.780530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Number of Params: 51.8M


# **Translate weight**

In [4]:
pytorch_model = custom_pytorch(num_classes)

pytorch_model.load_state_dict(torch.load('/kaggle/working/custom.pth', map_location=torch.device('cpu')))
pytorch_model.eval()

# Helper function to convert PyTorch tensors to NumPy arrays
def pt_to_np(tensor):
    return tensor.detach().cpu().numpy()

In [5]:
# Map PyTorch weights to TensorFlow model
tf_model.layers[1].set_weights([pt_to_np(pytorch_model.conv1.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv1.bias)])
tf_model.layers[2].set_weights([pt_to_np(pytorch_model.bn1.weight), pt_to_np(pytorch_model.bn1.bias),
                                pt_to_np(pytorch_model.bn1.running_mean), pt_to_np(pytorch_model.bn1.running_var)])

tf_model.layers[4].set_weights([pt_to_np(pytorch_model.conv2.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv2.bias)])
tf_model.layers[5].set_weights([pt_to_np(pytorch_model.bn2.weight), pt_to_np(pytorch_model.bn2.bias),
                                pt_to_np(pytorch_model.bn2.running_mean), pt_to_np(pytorch_model.bn2.running_var)])

tf_model.layers[7].set_weights([pt_to_np(pytorch_model.conv3.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv3.bias)])
tf_model.layers[8].set_weights([pt_to_np(pytorch_model.bn3.weight), pt_to_np(pytorch_model.bn3.bias),
                                pt_to_np(pytorch_model.bn3.running_mean), pt_to_np(pytorch_model.bn3.running_var)])

tf_model.layers[10].set_weights([pt_to_np(pytorch_model.conv4.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv4.bias)])
tf_model.layers[11].set_weights([pt_to_np(pytorch_model.bn4.weight), pt_to_np(pytorch_model.bn4.bias),
                                 pt_to_np(pytorch_model.bn4.running_mean), pt_to_np(pytorch_model.bn4.running_var)])

tf_model.layers[15].set_weights([pt_to_np(pytorch_model.fc1.weight.T), pt_to_np(pytorch_model.fc1.bias)])
tf_model.layers[17].set_weights([pt_to_np(pytorch_model.fc2.weight.T), pt_to_np(pytorch_model.fc2.bias)])

In [6]:
tf_model.save('custom.h5')

In [7]:
load_tf_model = models.load_model('/kaggle/working/custom.h5')

In [8]:
n_parameters = load_tf_model.count_params()
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 51.8M
